In [83]:
import json
import tqdm
import hashlib
from typing import (
    Union,
    Sequence
)
import math
import random
import sympy as sp
import glob


In [114]:

# function hub
def read_json_file(file_path): 
    ''' opens a json file, return its content save in a dictionary
    @input: file path to the json file
    @output: a dictionary of json file content
    '''
    file = open(file_path, 'r')
    values = json.load(file)
    file.close()
    return values

# main hash function
def hash_elems(*a):
    
    
    h = hashlib.sha256()
    h.update("|".encode("utf-8"))
    
    for x in a:

        if not x:
            # This case captures empty lists and None, nicely guaranteeing that we don't
            # need to do a recursive call if the list is empty. So we need a string to
            # feed in for both of these cases. "None" would be a Python-specific thing,
            # so we'll go with the more JSON-ish "null".
            hash_me = "null"

        elif isinstance(x, str):
            # strings are iterable, so it's important to handle them before the following check
            hash_me = x
        elif isinstance(x, Sequence):
             # The simplest way to deal with lists, tuples, and such are to crunch them recursively.
            hash_me = str(hash_elems(*x))           
        else:
            hash_me = str(x)
        h.update((hash_me + "|").encode("utf-8"))

    # Note: the returned value will range from [1,Q), because zeros are bad
    # for some of the nonces. (g^0 == 1, which would be an unhelpful thing
    # to multiply something with, if you were trying to encrypt it.)

    # Also, we don't need the checked version of int_to_q, because the
    # modulo operation here guarantees that we're in bounds.
    # return int_to_q_unchecked(
    #     1 + (int.from_bytes(h.digest(), byteorder="big") % Q_MINUS_ONE)
    # )

    return (int.from_bytes(h.digest(), byteorder="big") % (q - 1))



def is_valid_of_Z_boundary(n, boundary):
    is_valid = True
    if isinstance(n, str):
        n = int(n) 
    if n < 0 or n >= boundary:
        is_valid = False
 
    return is_valid   
    
def is_valid_of_Zr_boundary(n, boundary):
    is_valid = True
    if isinstance(n, str):
        n = int(n)    
    if n < 0 or n >= boundary:
        is_valid = False
    
    n_power = pow(n, q, boundary)
    if n_power != 1:
        is_valid = False
    return is_valid


def equals(a, b):
    ''''compares two values
    @input: two integers a, b 
    @output: True if a, b have same values, False otherwise
    '''
    return (a == b)

def get_length(num):
    ''' get the length of an integer using log
    @input: a positive integer
    @output: its length 
    '''
    len_of_num = int(math.log(num, 10) + 1)
    return len_of_num

def mod_p(n):
    if isinstance(n, str):
        n = int(n) 
    return n % p

def get_production(*targets):
    
    product = 1
    for target in targets:
        if isinstance(target, str):
            target = int(target) 
        product = mod_p(product * target)
    return product
    

# the following code is adapted from GeeksforGeeks
def power_mod(x, y, p): 
    ''' do modular exponentiation. 
    @input: x - base, 
            y - exponent,
            m - modulus
    @output: integer result of x^y % m
    '''
    # Initialize result 
    res = 1
      
    # Update x if it is more than or equal to p 
    if(x >= p):
        x = x % p

    # positive exponential
    while (y > 0): 
          
        # If y is odd, multiply x with result 
        if (y & 1): 
            res = (res * x) % p
  
        # y must be even now 
        y = y >> 1; # y = y/2 
        x = (x * x) % p
    return res
    
def __miller_test(d, num):
    ''' find a odd number of d such that num - 1 = d * 2^r
    @input: d - a odd number that num - 1 = d * 2^r for r >= 1
            num - the number needs to be check against
    @output: True if num is prime, False if it's a composite
    '''
    # Pick a random number in [2..n-2] 
    # Corner cases make sure that n > 4 
    a = 2 + random.randint(1, num - 4)

    # Compute a^d % n 
    x = power_mod(a, d, num)
  
    if (x == 1 or x == num - 1): 
        return True
  
    # Keep squaring x while one of the following doesn't happen 
    # (i) d does not reach n-1 
    # (ii) (x^2) % n is not 1 
    # (iii) (x^2) % n is not n-1 
    while (d != num - 1): 
        x = (x * x) % num
        d *= 2
  
        if (x == 1): 
            return False; 
        if (x == num - 1): 
            return True
  
    # Return composite 
    return False

def is_prime(num, k): 
    ''' implements Miller-Rabin algorithm to test the primality of a number
    @input: num - a positive integer
            k - the number of iterations, impacting accuracy
    @output: True if it's a prime, False otherwise 
    '''
    # Corner cases 
    if (num <= 1 or num == 4): 
        return False
    if (num <= 3): 
        return True
  
    # Find r such that n = 2^d * r + 1 for some r >= 1 
    d = num - 1
    while (d % 2 == 0): 
        d //= 2
  
    # Iterate given number of 'k' times 
    for i in range(k): 
        if (__miller_test(d, num) == False): 
            return False
  
    return True

def is_divisor(a, b): 
    '''check if a is a divisor of b
    @input: a, b - positive integers 
    @output: True if a is a divisor of b, False otherwise'''
    return (a % b == 0)

def create_hash_parameter_list(i): 
    '''create a list of parameters needed for hash computation
    @input: i - the i-th trustee/guardian
            base-hash - given base hash code
    @output: a list of all the parameters needed, including base hash Q, Ki,j, hi,j
    '''
    # invalid index input
    if (i < 0 or i > 4): 
        raise ValueError('i should be within range of 0 - 4.')

    else:
        # declare variables
        coefficient_file_path = ''
        param_list = []

        #append base hash to list
        #param_list.append(base_hash)

        # get file name dynamically
        coefficients_file_path = (coefficients_path + str(i) + '.json')
        
        
        # read file 
        coefficients = read_json_file(coefficients_file_path)

        # get all the commitment values Ki,j and append to list 
        commitments = coefficients['coefficientCommitments']
        for commitment in commitments: 
            param_list.append(commitment)
        
        # get all the hi,j values and append to list
        proofs = coefficients['coefficientProofs']
        for item in proofs:
            h_ij = item['h']
            param_list.append(h_ij)

        return param_list

    
def get_field_ij(i, j, field): 
    ''' getter methods to get ui,j, hi,j, ki,j
    @input: i - i th member, j - j th coefficient, field - u/h/k
    @output: a certain field of the i-th member, stored in its j-th item
    '''
    # exception
    if (i < 0 or i >= int(num_of_guardian) or j < 0 or j >= int(threshold)):
        raise ValueError('i should be in range (0,{}), j should be in range (0, {})'
                         .format(num_of_guardian, threshold))

    # get file name dynamically according to i
    coefficients_file_path = (coefficients_path + str(i) + '.json')
    
    coefficients = read_json_file(coefficients_file_path)
    proofs = coefficients['coefficient_proofs']

    # get specific value according to j and the given field
    return int(proofs[j][field])


### Green box 1.1 Baseline parameters - overview 

In [37]:
# TODO: needs to update paths to new results when the constants.json is given with non-null 


In [116]:

# file path configuration
constants_file_path = 'results/constants.json'
context_file_path = 'new_results/context.json'
ballot_file_path = 'new_results/encrypted_ballots/ballot_ballot-ce63a048-d67c-11ea-8412-acde48001122.json'
ballot_folder_path = 'new_results/encrypted_ballots/'
tally_file_path = 'new_results/tally.json'
coefficients_path = 'new_results/coefficients/coefficient_validation_set_hamilton-county-canvass-board-member-'


constants = read_json_file(constants_file_path)
context = read_json_file(context_file_path)
ballot = read_json_file(ballot_file_path)
tally = read_json_file(tally_file_path)

# basic parameters
g = 0
p = 0
q = 0
r = 0
gInv = 0
K = 0

In [92]:

#print(constants)

g = constants['g']
p = constants['p']
q = constants['q']
q_test = pow(2, 256) - 189

r = constants['r']
g_inv = constants['gInv']

#print('g:'+str(g))
#print('p:'+str(p))
#print('q:'+str(q))


#print(context)

K = int(context['elgamal_public_key'])
q_bar = context['crypto_extended_base_hash']

print(p)


1044388881413152506691752710716624382579964249047383780384233483283953907971553643537729993126875883902173634017777416360502926082946377942955704498542097614841825246773580689398386320439747911160897731551074903967243883427132918813748016269754522343505285898816777211761912392772914485521155521641049273446207578961939840619466145806859275053476560973295158703823395710210329314709715239251736552384080845836048778667318931418338422443891025911884723433084701207771901944593286624979917391350564662632723703007964229849154756196890615252286533089643184902706926081744149289517418249153634178342075381874131646013444796894582106870531535803666254579602632453103741452569793905551901541856173251385047414840392753585581909950158046256810542678368121278509960520957624737942914600310646609792665012858397381435755902851312071248102599442308951327039250818892493767423329663783709190716162023529669217300939783171415808233146823000766917789286154006042281423733706462905243774854543127239500245873582012

### Green box 1.2 Baseline parameters - check against each election

In [93]:

error = False 

# use Miller-Rabin algorithm to check the primality of p and q
# set iteration to run 50 times by default
DEFAULT_K = 50
if not is_prime(p, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(p, DEFAULT_K), var = "p"))
if not is_prime(q, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(q, DEFAULT_K), var = "q"))

# check equation p - 1 = qr
if not equals(p - 1, q * r): 
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = equals(p - 1, q * r), equation = "p - 1 = q * r"))

# check q is not a divisor of r, and 1 < g < p
if is_divisor(q, r):
    error = True
    print('''It\'s verfied as {res} that {var1} is not a divisor of {var2}.'''
      .format(res = not(is_divisor(q, r)), var1 = "q", var2 = "r"))

# check 1 < g < p
is_within_range = ((g > 1) and (g < p))
if not is_within_range: 
    error = True
    print('''It\'s verfied as {res} that {cond}.'''
      .format(res = is_within_range, cond = "1 < g < p"))

# check g^q mod p = 1 
result_actual = power_mod(g, q, p)
result_expected = 1 
if not equals(result_actual, result_expected):
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = str(equals(result_actual, result_expected)), 
              equation = 'g^q mod p = 1'))
if not error:
    print('success')


success


### Green box 2.1 Key Generation, confirm hash computation
calculate $c_i = H(Q,K_{i,0},K_{i,1},K_{i,2},...,K_{i, k-1},h_{i,0},h_{i,1},h_{i,2},...,h_{i,k-1})$ mod q

related file: context.json

coefficients files


In [94]:

# store variables of number of trustees/guardians, threshold, 
# base hash and extended hash
num_of_guardian = context['number_of_guardians']
threshold = context['quorum']
base_hash = context['crypto_base_hash']
extended_hash = context['crypto_extended_base_hash']


In [99]:

# save all members' hash results in a list
hash_res_list = []

for i in range(0, 5): 
    param_list = create_hash_parameter_list(i)
    #print(param_list)
    hash_res = hash_elems(q_bar, param_list) % q
    print('Guardian {i} hash: {hashcode}'.format(i = i, hashcode = hash_res))
    hash_res_list.append(hash_res)

# hash results mod q to generate ci 
ci_list = []

for i in range(0, 5):
    ci_i = hash_res_list[i] % q
    print('Guardian {i} ci : {ci}'.format(i = i, ci = ci_i))
    ci_list.append(ci_i)


Guardian 0 hash: 21992614219734542070698529642473783662753988303930712987276539480392282503571
Guardian 1 hash: 16632564400716230874345040373780486981979341329177429131302800852169549385149
Guardian 2 hash: 52877229097744776986253260648679325573876121044009194021684527207095222839403
Guardian 3 hash: 83975839686012003286504309217884602840598866105715397154736719455253374843178
Guardian 4 hash: 8214125560062964979105779754746792788918448730572255898745584163887585283754
Guardian 0 ci : 21992614219734542070698529642473783662753988303930712987276539480392282503571
Guardian 1 ci : 16632564400716230874345040373780486981979341329177429131302800852169549385149
Guardian 2 ci : 52877229097744776986253260648679325573876121044009194021684527207095222839403
Guardian 3 ci : 83975839686012003286504309217884602840598866105715397154736719455253374843178
Guardian 4 ci : 8214125560062964979105779754746792788918448730572255898745584163887585283754


### Green box 2.2 - check: $g^{u_{ij}} mod p = h_{i,j}K^{c_i}_{i,j} mod p$

In [105]:
# uses double for-loop to check, 0 <= i < 5, 0 <= j < 3
for i in range(0, num_of_guardian): 
    coefficients = read_json_file(coefficients_path+str(i)+'.json')
    for j in range(0, threshold): 
        coefficient_proofs = coefficients['coefficientProofs'][j]
        u_ij = int(coefficient_proofs['u'])
        h_ij = int(coefficient_proofs['h'])
        k_ij = int(coefficient_proofs['k'])
        
        #u_ij = get_field_ij(i, j, 'u')
        #h_ij = get_field_ij(i, j, 'h')
        #k_ij = get_field_ij(i, j, 'k')
        left = pow(g, u_ij, p)
        right = mod_p(mod_p(h_ij) * pow(k_ij, hash_res_list[i], p))
        print(left)
        print(right)
        print("--------------------------------------------------")
        
        #print('left {l}, \nright {r} '.format(l = left, r = right))
        print('It\'s verified as {res} that {equation} when {i_name} = {i_val}, {j_name} = {j_val}'
        .format(res = equals(left, right), 
                equation = '𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝=ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝',
                i_name = 'i',
                i_val = str(i),
                j_name = 'j',
                j_val = str(j)))
                

2111736491266413185292718549210384486514285944161182919606188064398015546176754346593639003825689324410084826460433753442959494494885371751426452789226839972620677998318399781836463723885361918603608154288613454862103095979451532298232500823927530840453195698023459572462925813198072685399377152224384022948794101833900570187960500933045460551209095929527747520089705005778094946766744307995993845315599679944766102918808343833864982463406774988391423558387118336016575997496372727992661250324136465161473844447911149833184423242484290248430184708228452603311238952585947692487377456207196407917293224715950875214868328739944435248232848038692664531983581090488134283482035564242332015444026654401605330529185566822475087223817360791933464704268961745085166161835169239490173478086318990291175284076055811283564883836719555148544899374075684141970128546910880548105370448127563047002351123533124282820130700527891053876566062869504311513612145665108936299610017607171974133816273659849808748992743679

8402602057408734380470802315970870922245111849376654952856885906430455414255690460369835299474302254778258110940531168100365121899246196417131909009602115549867152151998639632847203836379394563100407570954945560957951609884999526661531329835303016483014682316915799209019683961631201588924044859886407590814587780601678101438851608323789244935583847766344093030522602202390966962187321088686816454635766116719571383555036060396857431971615948515849849671221246175014525776353202740943461111582599311591187628062888092914230053485671177556467882015113481595330683380597015991243756472850090241950876351594896619423587169296787322717218938985259099451724625626565867939033942823150255410177511445188403079675058039579508575372293608562977042548185257171827418802923282574073907370181898981325347183011091278282474385634056164840936925898881206372549014891568787077463730295494543517399438022916183024111614847895260214163277667208734921366006638761455969832800443034514390186390901087016346274755104703

4329002966451187472675538439918550315742808426905467573314933895375912190350418137339529439253142865035762968776379162753250453846088029337424979164680818046633513661010573864019464586465910238337679925810714852142469279657190498066183788376003578186734148082289186391018721383491291574337804476445907190296387979537317740950372483598286811629108546817352027307816148117856862393961037576260321125812618292447030857880123717106600706886402816940762512959123821830322315724229574236360021115184960357766444434538497849326230545621187014459799594520568396403902886997536846017484737480585968462577521553360709355213043990362258566572606772236747257311744965655250044983373340583019875510565903606008186644915862689704867439568790912900878063025689965747885219699588056660159044358798880912981734793186645936705294886544126273793366306958228951045848870767925390639855152909155531044247090714482289646173267966551269878348192686817893516960012364644706589077106097930353981673827134646933112576926783649

NameError: name 'ballot' is not defined

In [85]:

# entire ballot files
big_alpha_list = []
big_beta_liat = []

for file in glob.glob(ballot_folder_path+"*.json"):
    
    ballot = read_json_file(file)
    error = False
    
    # green box 3
    contests = ballot['contests']
    for contest in contests:

        ballotSelections = contest['ballot_selections']
        for ballotSelection in ballotSelections:


            objectId = ballotSelection['object_id']

            alpha = int(ballotSelection['ciphertext']['pad'])
            beta = int(ballotSelection['ciphertext']['data'])
            big_alpha_list.append(alpha)
            big_beta_list.append(beta)

            a0 = int(ballotSelection['proof']['proof_zero_pad'])
            a1 = int(ballotSelection['proof']['proof_one_pad'])
            b0 = int(ballotSelection['proof']['proof_zero_data'])
            b1 = int(ballotSelection['proof']['proof_one_data'])
            c0 = int(ballotSelection['proof']['proof_zero_challenge'])
            c1 = int(ballotSelection['proof']['proof_one_challenge'])
            v0 = int(ballotSelection['proof']['proof_zero_response'])
            v1 = int(ballotSelection['proof']['proof_one_response'])

            alphaPow = pow(alpha, q, p)
            betaPow = pow(beta, q, p)
            a0Pow = pow(a0, q, p)
            b0Pow = pow(b0, q, p)
            a1Pow = pow(a1, q, p)
            b1Pow = pow(b1, q, p)

            #print("a0:"+str(a0))
            #print("alpha_1:"+str(alpha_1))
            # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.


            if not is_valid_of_Zrp(alpha) or \
               not is_valid_of_Zrp(beta) or \
               not is_valid_of_Zrp(a0) or \
               not is_valid_of_Zrp(a1) or \
               not is_valid_of_Zrp(b0) or \
               not is_valid_of_Zrp(b1):
                print ("alpha, beta, a0, a1, b0, b1, one of them is out of range.")
                error = True  

            # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
            # tempString = str(q_bar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
            # tempList = (q_bar, (alpha, beta), (a0, b0), (a1, b1))
            c = hash_elems(q_bar, alpha, beta, a0, b0, a1, b1)
            #print(c)

            #print((c0 + c1) % q)

            if c != (c0 + c1) % q:
                print("c != (c0 + c1) % q.")
                error = True

            # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
            if not is_valid_of_Z_boundary(c0, q) or \
               not is_valid_of_Z_boundary(c1, q) or \
               not is_valid_of_Z_boundary(v0, q) or \
               not is_valid_of_Z_boundary(v1, q):
                print("c0, c1, v0, v1, one of them is out of range.")
                error = True               

            # 1st equation: gv0=a0alphac0 mod p
            g_v0_ls = pow(g, v0, p)
            g_v0_rs = mod_p(a0 * pow(alpha, c0, p))

            # 2nd equation: gv1=a1alphac1 mod p
            g_v1_ls = pow(g, v1, p)
            g_v1_rs = mod_p(a1 * pow(alpha, c1, p))

            # 3rd equation: Kv0=b0betac0 mod p
            K_v0_ls = pow(K, v0, p)
            K_v0_rs = mod_p(b0 * pow(beta, c0, p))

            # 4th equation: gc1Kv1=b1betac1 mod p
            fourth_ls = mod_p(pow(g, c1, p) * pow(K, v1, p))
            fourth_rs = mod_p(b1 * pow(beta, c1, p))

            # checking
            if not equals(g_v0_ls,g_v0_rs):
                print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
                error = True
            if not equals(g_v1_ls, g_v1_rs):
                print("g_v1 != a_1*alpha^c_1")
                error = True
            if not equals(K_v0_ls, K_v0_rs):
                print("K_v0 != b_0*alpha^c_0")
                error = True
            if not equals(fourth_ls, fourth_rs):
                print("g_c1 * K_v1 != (b1 * beta^c1) % p")
                error = True


    if not error:
        print("[green box 3 success] "+file)  
    
    # green box 4
    for contest in contests:

        # total parameters
        big_a = int(contest['proof']['pad'])
        big_b = int(contest['proof']['data'])
        big_c = int(contest['proof']['challenge'])
        big_v = int(contest['proof']['response'])
        big_constant = int(contest['proof']['constant'])
        big_alpha = 1
        big_beta = 1

        if not is_valid_of_Z_boundary(big_v, q) or \
           not is_valid_of_Zr_boundary(big_a, p) or \
           not is_valid_of_Zr_boundary(big_b, p):
            error = True
            print("big_a, big_b, big_v, one of them is out of range.")

        ballotSelections = contest['ballot_selections']
        for ballotSelection in ballotSelections:

            alpha = int(ballotSelection['ciphertext']['pad'])
            beta = int(ballotSelection['ciphertext']['data'])  
            big_alpha = mod_p(big_alpha * alpha)
            big_beta = mod_p(big_beta * beta)

        temp_c = hash_elems(q_bar, big_alpha, big_beta, big_a, big_b)
        #print(temp_c)
        #print(big_c)


        # g power v mod p = a * A power c mod p
        g_v_ls = pow(g, big_v, p)
        a_ac_rs = mod_p((mod_p(big_a)) * pow(big_alpha, big_c, p))

        #print(g_v_ls)
        #print(a_ac_rs)
        if g_v_ls != a_ac_rs:
            error = True
            print("g_v_ls is not equal a_ac_rs.")


        if big_constant is None:
            constant_q = 0
        else:
            constant_q = big_constant

        # g power Lc mod p * K power v mod p = b * B power c mod p
        g_l_k_ls = mod_p(pow(g, mod_p(constant_q * big_c), p) * pow(K, big_v, p))
        b_bc_rs = mod_p(mod_p(big_b) * pow(big_beta, big_c, p))


        # mult_p(g_pow_p(mult_p(c, constant_q)), pow_p(k, v))
        # mult_p(b, pow_p(beta, c))
        #print(g_l_k_ls)
        #print(b_bc_rs)
        if g_l_k_ls != b_bc_rs:
            error = True
            print("g_l_k_ls is not equal b_bc_rs.")        

    if not error:
        print("[green box 4 success] "+file)
    
    

[success] new_results/encrypted_ballots/ballot_ballot-ce65aa50-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce66267e-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce658e12-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce644570-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce68fa66-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce67b412-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce663358-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce690baa-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce671750-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce694df4-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/

[success] new_results/encrypted_ballots/ballot_ballot-ce670616-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce67e20c-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce63d0ae-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce6671c4-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce6651d0-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce659c68-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce69da6c-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce69fa60-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce648a8a-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/ballot_ballot-ce67cfce-d67c-11ea-8412-acde48001122.json
[success] new_results/encrypted_ballots/

In [110]:





# one sample data
ballot = read_json_file(ballot_file_path)

error = False

contests = ballot['contests']
for contest in contests:
    
    ballotSelections = contest['ballot_selections']
    for ballotSelection in ballotSelections:


        objectId = ballotSelection['object_id']

        alpha = int(ballotSelection['ciphertext']['pad'])
        beta = int(ballotSelection['ciphertext']['data'])

        a0 = int(ballotSelection['proof']['proof_zero_pad'])
        a1 = int(ballotSelection['proof']['proof_one_pad'])
        b0 = int(ballotSelection['proof']['proof_zero_data'])
        b1 = int(ballotSelection['proof']['proof_one_data'])
        c0 = int(ballotSelection['proof']['proof_zero_challenge'])
        c1 = int(ballotSelection['proof']['proof_one_challenge'])
        v0 = int(ballotSelection['proof']['proof_zero_response'])
        v1 = int(ballotSelection['proof']['proof_one_response'])

        alphaPow = pow(alpha, q, p)
        betaPow = pow(beta, q, p)
        a0Pow = pow(a0, q, p)
        b0Pow = pow(b0, q, p)
        a1Pow = pow(a1, q, p)
        b1Pow = pow(b1, q, p)

        #print("a0:"+str(a0))
        #print("alpha_1:"+str(alpha_1))
        # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.
        
        
        if (not is_valid_of_Zrp(alpha) or not is_valid_of_Zrp(beta) or not is_valid_of_Zrp(a0) 
            or not is_valid_of_Zrp(a1) or not is_valid_of_Zrp(b0) or not is_valid_of_Zrp(b1)):
            print ("alpha, beta, a0, a1, b0, b1, one of them is out of range.")
            error = True  

        # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
        # tempString = str(q_bar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
        # tempList = (q_bar, (alpha, beta), (a0, b0), (a1, b1))
        c = hash_elems(q_bar, alpha, beta, a0, b0, a1, b1)
        #print(c)
        
        #print((c0 + c1) % q)
        
        if c != (c0 + c1) % q:
            print("c != (c0 + c1) % q.")
            error = True

        # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
        if not is_valid_of_Z_boundary(c0, q) or \
           not is_valid_of_Z_boundary(c1, q) or \
           not is_valid_of_Z_boundary(v0, q) or \
           not is_valid_of_Z_boundary(v1, q):
            print("c0, c1, v0, v1, one of them is out of range.")
            error = True               

        # 1st equation: gv0=a0alphac0 mod p
        g_v0_ls = pow(g, v0, p)
        g_v0_rs = mod_p(a0 * pow(alpha, c0, p))
    
        # 2nd equation: gv1=a1alphac1 mod p
        g_v1_ls = pow(g, v1, p)
        g_v1_rs = mod_p(a1 * pow(alpha, c1, p))

        # 3rd equation: Kv0=b0betac0 mod p
        K_v0_ls = pow(K, v0, p)
        K_v0_rs = mod_p(b0 * pow(beta, c0, p))
    
        # 4th equation: gc1Kv1=b1betac1 mod p
        fourth_ls = mod_p(pow(g, c1, p) * pow(K, v1, p))
        fourth_rs = mod_p(b1 * pow(beta, c1, p))

        # checking
        if not equals(g_v0_ls,g_v0_rs):
            print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
            error = True
        if not equals(g_v1_ls, g_v1_rs):
            print("g_v1 != a_1*alpha^c_1")
            error = True
        if not equals(K_v0_ls, K_v0_rs):
            print("K_v0 != b_0*alpha^c_0")
            error = True
        if not equals(fourth_ls, fourth_rs):
            print("g_c1 * K_v1 != (b1 * beta^c1) % p")
            error = True

                                               
if not error:
    print("success")   



success


In [82]:
# Green box 4


for file in glob.glob(ballot_folder_path+"*.json"):
    
    ballot = read_json_file(file)

    error = False

    contests = ballot['contests']
    for contest in contests:

        # total parameters
        big_a = int(contest['proof']['pad'])
        big_b = int(contest['proof']['data'])
        big_c = int(contest['proof']['challenge'])
        big_v = int(contest['proof']['response'])
        big_constant = int(contest['proof']['constant'])
        big_alpha = 1
        big_beta = 1

        if not is_valid_of_Z_boundary(big_v, q) or \
           not is_valid_of_Zr_boundary(big_a, p) or \
           not is_valid_of_Zr_boundary(big_b, p):
            error = True
            print("big_a, big_b, big_v, one of them is out of range.")

        ballotSelections = contest['ballot_selections']
        for ballotSelection in ballotSelections:

            alpha = int(ballotSelection['ciphertext']['pad'])
            beta = int(ballotSelection['ciphertext']['data'])  
            big_alpha = mod_p(big_alpha * alpha)
            big_beta = mod_p(big_beta * beta)

        temp_c = hash_elems(q_bar, big_alpha, big_beta, big_a, big_b)
        #print(temp_c)
        #print(big_c)


        # g power v mod p = a * A power c mod p
        g_v_ls = pow(g, big_v, p)
        a_ac_rs = mod_p((mod_p(big_a)) * pow(big_alpha, big_c, p))

        #print(g_v_ls)
        #print(a_ac_rs)
        if g_v_ls != a_ac_rs:
            error = True
            print("g_v_ls is not equal a_ac_rs.")


        if big_constant is None:
            constant_q = 0
        else:
            constant_q = big_constant

        # g power Lc mod p * K power v mod p = b * B power c mod p
        g_l_k_ls = mod_p(pow(g, mod_p(constant_q * big_c), p) * pow(K, big_v, p))
        b_bc_rs = mod_p(mod_p(big_b) * pow(big_beta, big_c, p))


        # mult_p(g_pow_p(mult_p(c, constant_q)), pow_p(k, v))
        # mult_p(b, pow_p(beta, c))
        #print(g_l_k_ls)
        #print(b_bc_rs)
        if g_l_k_ls != b_bc_rs:
            error = True
            print("g_l_k_ls is not equal b_bc_rs.")        

    if not error:
        print("success")   

success


In [69]:
context = read_json_file("../data_08042020/context.json")

FileNotFoundError: [Errno 2] No such file or directory: '../data_08042020/context.json'

In [117]:
# green box 6 

# get all the contest names stored in the first layer
contest_names = list(tally.keys())

# loop over each contest 
for contest_name in contest_names: 
    error = False
    print(contest_name)
    
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        print("     " + selection_name)
        selection = tally[contest_name]['selections'][selection_name]
        shares = selection['shares']
        big_A = int(selection['message']['pad'])
        big_B = int(selection['message']['data'])
        
        # loop over each guardian's share of decryption
        for i in range(len(shares)):
            share = shares[i]
            M_i = int(share['share'])
            proof = share['proof']
            v_i = int(proof['response'])
            a_i = int(proof['pad'])
            b_i = int(proof['data'])
            c_i = int(proof['challenge'])
            k_i = int(get_field_ij(i, 0, 'public_key'))
            
            # calculate using given values
            c_i_expected = hash_elems(q_bar, big_A, big_B, a_i, b_i, M_i)
            #print("actual c   " + str(c_i))
            #print("expected c " + str(c_i_expected))
            equ_1_left = pow(g, v_i, p)
            equ_1_right = mod_p(mod_p(a_i) * pow(k_i, c_i, p))
            equ_2_left = pow(big_A, v_i, p)
            equ_2_right = mod_p(mod_p(b_i) * pow(M_i, c_i, p))
            #print("equation1 left   " + str(equ_1_left))
            #print("equation1 right   " + str(equ_1_right))
            
            # confirm that v_i is in the set of Z_q
            if not is_valid_of_Z_boundary(v_i, q):
                error = True
                print("       v_i error " + str(error))
                
            # confirm that a_i and b_i are both in the set Z_r_q
            if not is_valid_of_Zr_boundary(a_i, q):
                error = True
                print("       a_i error " + str(error))
            if not is_valid_of_Zr_boundary(b_i, q):
                error = True
                print("       b_i error " + str(error))
                
            # confirm challege value c_i = H(q_bar, (A,B), (a_i, b_i), M_i)
            if not equals(c_i, c_i_expected):
                error = True
                print("       c_i error " + str(error))
                
            # check first equation, g^v_i = a_i * K_i^c_i
            if not equals(equ_1_left, equ_1_right):
                error = True
                print("       equation1 error " + str(error))
                
            # check second equation, A^v_i = b_i * M_i^c_i
            if not equals(equ_2_left, equ_2_right):
                error = True
                print("       equation2 error " + str(error))
                
            
        
if not error:
    print("success")

president-vice-president-contest
     barchi-hallaren-selection


ValueError: pow() 3rd argument cannot be 0

In [21]:
# TODO: verify big_A, big_B by getting products from 100 ballot files

In [75]:
# green box 9 
error = False

for contest_name in contest_names: 
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    #print(contest_name) 
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        #print("     " + selection_name)
        m_product = 1
        selection = tally[contest_name]['selections'][selection_name]
        shares = selection['shares']
        big_B = int(selection['message']['data'])
        big_M = int(selection['value'])
        t = int(selection['tally']) 
        
        for share in shares: 
            M_i = int(share['share'])
            m_product *= mod_p(M_i)
            
        # check equation B = M * (M_i products) mod p
        if not equals(mod_p(big_B), mod_p(big_M * m_product)):
            error = True
            print(error)
            
        # check equation M = g^t mod p
        if not equals(big_M, pow(g, t, p)):
            error = True
            print(error)
if not error:
    print("success")

success
